In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# To fix transactions to add stores with no previous values as 0 values
import janitor

In [2]:
test = pd.read_csv('test.csv', parse_dates=['date'])
sample = pd.read_csv('sample_submission.csv')
train = pd.read_csv('train.csv', parse_dates=['date'])
holidays = pd.read_csv('holidays_events.csv', parse_dates=['date'])
oil = pd.read_csv('oil.csv', parse_dates=['date'])
stores = pd.read_csv('stores.csv')
transactions = pd.read_csv('transactions.csv', parse_dates=['date'])

To set up date ranges for the train and test dataframes. 

In [258]:
train_start = train.date.min().date()
train_end = train.date.max().date()
num_train_date = train.date.nunique()
train_len = (train_end - train_start).days + 1
test_start = test.date.min().date()
test_end = test.date.max().date()
num_test_date = test.date.nunique()

Items to look at:

Move transactions into train by store number and dates

Figure out how to deal with Transferred in holidays

Figure out how to link holidays by store; just one location for local, which locations for regional, and all locations for national

In [259]:
test

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0
...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1
28508,3029396,2017-08-31,9,PREPARED FOODS,0
28509,3029397,2017-08-31,9,PRODUCE,1
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9


In [260]:
test.dtypes

id                      int64
date           datetime64[ns]
store_nbr               int64
family                 object
onpromotion             int64
dtype: object

In [261]:
train

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0
1,1,2013-01-01,1,BABY CARE,0.000,0
2,2,2013-01-01,1,BEAUTY,0.000,0
3,3,2013-01-01,1,BEVERAGES,0.000,0
4,4,2013-01-01,1,BOOKS,0.000,0
...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8


In [262]:
train.dtypes

id                      int64
date           datetime64[ns]
store_nbr               int64
family                 object
sales                 float64
onpromotion             int64
dtype: object

In [263]:
train['family'].unique()

array(['AUTOMOTIVE', 'BABY CARE', 'BEAUTY', 'BEVERAGES', 'BOOKS',
       'BREAD/BAKERY', 'CELEBRATION', 'CLEANING', 'DAIRY', 'DELI', 'EGGS',
       'FROZEN FOODS', 'GROCERY I', 'GROCERY II', 'HARDWARE',
       'HOME AND KITCHEN I', 'HOME AND KITCHEN II', 'HOME APPLIANCES',
       'HOME CARE', 'LADIESWEAR', 'LAWN AND GARDEN', 'LINGERIE',
       'LIQUOR,WINE,BEER', 'MAGAZINES', 'MEATS', 'PERSONAL CARE',
       'PET SUPPLIES', 'PLAYERS AND ELECTRONICS', 'POULTRY',
       'PREPARED FOODS', 'PRODUCE', 'SCHOOL AND OFFICE SUPPLIES',
       'SEAFOOD'], dtype=object)

In [264]:
test['family'].unique()

array(['AUTOMOTIVE', 'BABY CARE', 'BEAUTY', 'BEVERAGES', 'BOOKS',
       'BREAD/BAKERY', 'CELEBRATION', 'CLEANING', 'DAIRY', 'DELI', 'EGGS',
       'FROZEN FOODS', 'GROCERY I', 'GROCERY II', 'HARDWARE',
       'HOME AND KITCHEN I', 'HOME AND KITCHEN II', 'HOME APPLIANCES',
       'HOME CARE', 'LADIESWEAR', 'LAWN AND GARDEN', 'LINGERIE',
       'LIQUOR,WINE,BEER', 'MAGAZINES', 'MEATS', 'PERSONAL CARE',
       'PET SUPPLIES', 'PLAYERS AND ELECTRONICS', 'POULTRY',
       'PREPARED FOODS', 'PRODUCE', 'SCHOOL AND OFFICE SUPPLIES',
       'SEAFOOD'], dtype=object)

In [265]:
transactions

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922
...,...,...,...
83483,2017-08-15,50,2804
83484,2017-08-15,51,1573
83485,2017-08-15,52,2255
83486,2017-08-15,53,932


To use the janitor function to add 0 values for transactions for stores that don't have any transactions listed on a particular date. This will make it possible to merge transactions into train_test dataframe.

In [266]:
transactions = transactions.complete('date', 'store_nbr').fillna(0, downcast='infer')

In [267]:
transactions

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-01,1,0
2,2013-01-01,2,0
3,2013-01-01,3,0
4,2013-01-01,4,0
...,...,...,...
90823,2017-08-15,29,1302
90824,2017-08-15,21,1059
90825,2017-08-15,42,995
90826,2017-08-15,22,766


In [268]:
sample

,id,sales
0,3000888,0.0
1,3000889,0.0
2,3000890,0.0
3,3000891,0.0
4,3000892,0.0
...,...,...
28507,3029395,0.0
28508,3029396,0.0
28509,3029397,0.0
28510,3029398,0.0


In [269]:
oil

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
...,...,...
1213,2017-08-25,47.65
1214,2017-08-28,46.40
1215,2017-08-29,46.46
1216,2017-08-30,45.96


To look at the missing oil dates. And it seems the number of those missing dates match the number of weekends in the the overall time period. 

In [270]:
missing_oil_dates = pd.date_range(train_start, test_end).difference(oil.date)
num_missing_oil_dates = len(missing_oil_dates)
print(num_missing_oil_dates)

486


In [271]:
num_wknd_missing = (missing_oil_dates.weekday >= 5).sum()
print(num_wknd_missing)

486


In [272]:
total_num_wknd = (pd.date_range(train_start, test_end).weekday >= 5).sum()
print(total_num_wknd)

486


Because grocery stores open on weekends as well, there is a need to estimate the missing oil prices if they are to be used in the models. For simplicity, the missing values in the oil column are filled using linear interpolation, where the estimated value is assumed to lie on the line joining the nearest values to the left and right.

In [273]:
oil = oil.merge(
    pd.DataFrame({"date": pd.date_range(train_start, test_end)}),
    on="date",
    how="outer",
).sort_values("date", ignore_index=True)

oil = oil.interpolate(method="pad", limit_direction="forward")

In [274]:
oil

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-05,93.12
...,...,...
1699,2017-08-27,47.65
1700,2017-08-28,46.40
1701,2017-08-29,46.46
1702,2017-08-30,45.96


In [275]:
holidays

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
...,...,...,...,...,...,...
345,2017-12-22,Additional,National,Ecuador,Navidad-3,False
346,2017-12-23,Additional,National,Ecuador,Navidad-2,False
347,2017-12-24,Additional,National,Ecuador,Navidad-1,False
348,2017-12-25,Holiday,National,Ecuador,Navidad,False


To remove the holidays where Transfer is True, since they are not really actual celebrated holidays

In [276]:
holidays['type'].value_counts()

Holiday       221
Event          56
Additional     51
Transfer       12
Bridge          5
Work Day        5
Name: type, dtype: int64

In [277]:
holidays = holidays[holidays.transferred.eq(False)]

In [278]:
holidays

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
...,...,...,...,...,...,...
345,2017-12-22,Additional,National,Ecuador,Navidad-3,False
346,2017-12-23,Additional,National,Ecuador,Navidad-2,False
347,2017-12-24,Additional,National,Ecuador,Navidad-1,False
348,2017-12-25,Holiday,National,Ecuador,Navidad,False


In [279]:
stores

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4
5,6,Quito,Pichincha,D,13
6,7,Quito,Pichincha,D,8
7,8,Quito,Pichincha,D,8
8,9,Quito,Pichincha,B,6
9,10,Quito,Pichincha,C,15


In [280]:
pd.isnull(test).sum()[pd.isnull(test).sum() > 0]

Series([], dtype: int64)

In [281]:
pd.isnull(train).sum()[pd.isnull(train).sum() > 0]

Series([], dtype: int64)

In [282]:
pd.isnull(transactions).sum()[pd.isnull(transactions).sum() > 0]

Series([], dtype: int64)

In [283]:
pd.isnull(oil).sum()[pd.isnull(oil).sum() > 0]

dcoilwtico    1
dtype: int64

In [284]:
pd.isnull(holidays).sum()[pd.isnull(holidays).sum() > 0]

Series([], dtype: int64)

In [285]:
pd.isnull(stores).sum()[pd.isnull(stores).sum() > 0]

Series([], dtype: int64)

To change the oil NaN to 0.

In [286]:
oil = oil.fillna(0)

In [287]:
pd.isnull(oil).sum()[pd.isnull(oil).sum() > 0]

Series([], dtype: int64)

To merge train and test just for initial data manipulation.

In [288]:
train_test = pd.concat([train, test], ignore_index=True)

In [289]:
train_test

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0
...,...,...,...,...,...,...
3029395,3029395,2017-08-31,9,POULTRY,NaN,1
3029396,3029396,2017-08-31,9,PREPARED FOODS,NaN,0
3029397,3029397,2017-08-31,9,PRODUCE,NaN,1
3029398,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,NaN,9


In [290]:
train_test.dtypes

id                      int64
date           datetime64[ns]
store_nbr               int64
family                 object
sales                 float64
onpromotion             int64
dtype: object

To create a day of the week column for the dataframe and convert to numbers. 

In [291]:
train_test['day_of_week'] = train_test['date'].dt.day_name()
mapping = {
    'Sunday': 7,
    'Monday': 1,
    'Tuesday': 2,
    'Wednesday': 3,
    'Thursday' : 4,
    'Friday' : 5,
    'Saturday' : 6
}
train_test['day_of_week'] = train_test['day_of_week'].replace(mapping)

To create a weekend column to denote whether the day of the week is a weekend or not. 

In [292]:
train_test['weekend'] = pd.Series(0, index=train_test.index).mask(train_test['day_of_week']>5, 1)

To create a month number column. 

In [293]:
train_test['month'] = pd.DatetimeIndex(train_test['date']).month

To add the year and the day and week of year. 

In [294]:
train_test['day_of_year'] = train_test['date'].dt.dayofyear
train_test['week_of_year'] = train_test['date'].dt.isocalendar().week
train_test['year'] = train_test['date'].dt.year

In [295]:
train_test

,id,date,store_nbr,family,sales,onpromotion,day_of_week,weekend,month,day_of_year,week_of_year,year
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,2,0,1,1,1,2013
1,1,2013-01-01,1,BABY CARE,0.0,0,2,0,1,1,1,2013
2,2,2013-01-01,1,BEAUTY,0.0,0,2,0,1,1,1,2013
3,3,2013-01-01,1,BEVERAGES,0.0,0,2,0,1,1,1,2013
4,4,2013-01-01,1,BOOKS,0.0,0,2,0,1,1,1,2013
...,...,...,...,...,...,...,...,...,...,...,...,...
3029395,3029395,2017-08-31,9,POULTRY,NaN,1,4,0,8,243,35,2017
3029396,3029396,2017-08-31,9,PREPARED FOODS,NaN,0,4,0,8,243,35,2017
3029397,3029397,2017-08-31,9,PRODUCE,NaN,1,4,0,8,243,35,2017
3029398,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,NaN,9,4,0,8,243,35,2017


To merge the oil values into the train_test dataframe. For some reason it added 3 additional rows on the bottom, so these were eliminated by looking for those rows where id doesn't exist. Also, for some reason it created NaNs for dates where there were no oil production, so those had to be eliminated. 

In [296]:
train_test = pd.merge(train_test, oil, on='date', how='outer')

In [297]:
train_test = train_test[train_test['id'].notna()]
train_test['dcoilwtico'].fillna(0)

0           0.00
1           0.00
2           0.00
3           0.00
4           0.00
           ...  
3029395    47.26
3029396    47.26
3029397    47.26
3029398    47.26
3029399    47.26
Name: dcoilwtico, Length: 3029400, dtype: float64

In [298]:
train_test

,id,date,store_nbr,family,sales,onpromotion,day_of_week,weekend,month,day_of_year,week_of_year,year,dcoilwtico
0,0.0,2013-01-01,1.0,AUTOMOTIVE,0.0,0.0,2.0,0.0,1.0,1.0,1,2013.0,0.00
1,1.0,2013-01-01,1.0,BABY CARE,0.0,0.0,2.0,0.0,1.0,1.0,1,2013.0,0.00
2,2.0,2013-01-01,1.0,BEAUTY,0.0,0.0,2.0,0.0,1.0,1.0,1,2013.0,0.00
3,3.0,2013-01-01,1.0,BEVERAGES,0.0,0.0,2.0,0.0,1.0,1.0,1,2013.0,0.00
4,4.0,2013-01-01,1.0,BOOKS,0.0,0.0,2.0,0.0,1.0,1.0,1,2013.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3029395,3029395.0,2017-08-31,9.0,POULTRY,NaN,1.0,4.0,0.0,8.0,243.0,35,2017.0,47.26
3029396,3029396.0,2017-08-31,9.0,PREPARED FOODS,NaN,0.0,4.0,0.0,8.0,243.0,35,2017.0,47.26
3029397,3029397.0,2017-08-31,9.0,PRODUCE,NaN,1.0,4.0,0.0,8.0,243.0,35,2017.0,47.26
3029398,3029398.0,2017-08-31,9.0,SCHOOL AND OFFICE SUPPLIES,NaN,9.0,4.0,0.0,8.0,243.0,35,2017.0,47.26


For some reason this outer join converted the integer values to floating values, so these needed to be reinstated. 

In [299]:
train_test['id']=train_test['id'].astype(np.int64)
train_test['store_nbr']=train_test['store_nbr'].astype(np.int64)
train_test['onpromotion']=train_test['onpromotion'].astype(np.int64)

In [300]:
train_test

,id,date,store_nbr,family,sales,onpromotion,day_of_week,weekend,month,day_of_year,week_of_year,year,dcoilwtico
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,2.0,0.0,1.0,1.0,1,2013.0,0.00
1,1,2013-01-01,1,BABY CARE,0.0,0,2.0,0.0,1.0,1.0,1,2013.0,0.00
2,2,2013-01-01,1,BEAUTY,0.0,0,2.0,0.0,1.0,1.0,1,2013.0,0.00
3,3,2013-01-01,1,BEVERAGES,0.0,0,2.0,0.0,1.0,1.0,1,2013.0,0.00
4,4,2013-01-01,1,BOOKS,0.0,0,2.0,0.0,1.0,1.0,1,2013.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3029395,3029395,2017-08-31,9,POULTRY,NaN,1,4.0,0.0,8.0,243.0,35,2017.0,47.26
3029396,3029396,2017-08-31,9,PREPARED FOODS,NaN,0,4.0,0.0,8.0,243.0,35,2017.0,47.26
3029397,3029397,2017-08-31,9,PRODUCE,NaN,1,4.0,0.0,8.0,243.0,35,2017.0,47.26
3029398,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,NaN,9,4.0,0.0,8.0,243.0,35,2017.0,47.26


To merge the stores dataframe into the train_test dataframe.

In [301]:
train_test = pd.merge(train_test, stores, on='store_nbr', how='outer')

In [302]:
train_test

,id,date,store_nbr,family,sales,onpromotion,day_of_week,weekend,month,day_of_year,week_of_year,year,dcoilwtico,city,state,type,cluster
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,2.0,0.0,1.0,1.0,1,2013.0,0.00,Quito,Pichincha,D,13
1,1,2013-01-01,1,BABY CARE,0.0,0,2.0,0.0,1.0,1.0,1,2013.0,0.00,Quito,Pichincha,D,13
2,2,2013-01-01,1,BEAUTY,0.0,0,2.0,0.0,1.0,1.0,1,2013.0,0.00,Quito,Pichincha,D,13
3,3,2013-01-01,1,BEVERAGES,0.0,0,2.0,0.0,1.0,1.0,1,2013.0,0.00,Quito,Pichincha,D,13
4,4,2013-01-01,1,BOOKS,0.0,0,2.0,0.0,1.0,1.0,1,2013.0,0.00,Quito,Pichincha,D,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3029395,3029395,2017-08-31,9,POULTRY,NaN,1,4.0,0.0,8.0,243.0,35,2017.0,47.26,Quito,Pichincha,B,6
3029396,3029396,2017-08-31,9,PREPARED FOODS,NaN,0,4.0,0.0,8.0,243.0,35,2017.0,47.26,Quito,Pichincha,B,6
3029397,3029397,2017-08-31,9,PRODUCE,NaN,1,4.0,0.0,8.0,243.0,35,2017.0,47.26,Quito,Pichincha,B,6
3029398,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,NaN,9,4.0,0.0,8.0,243.0,35,2017.0,47.26,Quito,Pichincha,B,6


To merge the transactions dataframe into the train_test dataframe on both date and store number since the same transactions need to be show by store by date for all the different product sales for each day and store. 

In [303]:
train_test = pd.merge(train_test, transactions, on=['date', 'store_nbr'], how='outer')

In [304]:
train_test['transactions'].fillna(0, inplace=True)

In [305]:
train_test

,id,date,store_nbr,family,sales,onpromotion,day_of_week,weekend,month,day_of_year,week_of_year,year,dcoilwtico,city,state,type,cluster,transactions
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,2.0,0.0,1.0,1.0,1,2013.0,0.00,Quito,Pichincha,D,13,0.0
1,1,2013-01-01,1,BABY CARE,0.0,0,2.0,0.0,1.0,1.0,1,2013.0,0.00,Quito,Pichincha,D,13,0.0
2,2,2013-01-01,1,BEAUTY,0.0,0,2.0,0.0,1.0,1.0,1,2013.0,0.00,Quito,Pichincha,D,13,0.0
3,3,2013-01-01,1,BEVERAGES,0.0,0,2.0,0.0,1.0,1.0,1,2013.0,0.00,Quito,Pichincha,D,13,0.0
4,4,2013-01-01,1,BOOKS,0.0,0,2.0,0.0,1.0,1.0,1,2013.0,0.00,Quito,Pichincha,D,13,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3029395,3029395,2017-08-31,9,POULTRY,NaN,1,4.0,0.0,8.0,243.0,35,2017.0,47.26,Quito,Pichincha,B,6,0.0
3029396,3029396,2017-08-31,9,PREPARED FOODS,NaN,0,4.0,0.0,8.0,243.0,35,2017.0,47.26,Quito,Pichincha,B,6,0.0
3029397,3029397,2017-08-31,9,PRODUCE,NaN,1,4.0,0.0,8.0,243.0,35,2017.0,47.26,Quito,Pichincha,B,6,0.0
3029398,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,NaN,9,4.0,0.0,8.0,243.0,35,2017.0,47.26,Quito,Pichincha,B,6,0.0


In planning how to merge the holidays dataframe into the train_test dataframe, the list of unique values was run. Then it appears there are 69 rows where the dates are duplicated. Finally, in preparation for the merge, the locale column name was changed to holiday. 

In [306]:
print(holidays.apply(lambda col: col.unique()))

date           [2012-03-02T00:00:00.000000000, 2012-04-01T00:...
type           [Holiday, Transfer, Additional, Bridge, Work D...
locale                               [Local, Regional, National]
locale_name    [Manta, Cotopaxi, Cuenca, Libertad, Riobamba, ...
description    [Fundacion de Manta, Provincializacion de Coto...
transferred                                              [False]
dtype: object


In [307]:
holidays[holidays.duplicated(['date'], keep=False)]

,date,type,locale,locale_name,description,transferred
7,2012-06-25,Holiday,Regional,Imbabura,Provincializacion de Imbabura,False
8,2012-06-25,Holiday,Local,Latacunga,Cantonizacion de Latacunga,False
9,2012-06-25,Holiday,Local,Machala,Fundacion de Machala,False
10,2012-07-03,Holiday,Local,Santo Domingo,Fundacion de Santo Domingo,False
11,2012-07-03,Holiday,Local,El Carmen,Cantonizacion de El Carmen,False
...,...,...,...,...,...,...
319,2017-07-03,Holiday,Local,Santo Domingo,Fundacion de Santo Domingo,False
341,2017-12-08,Holiday,Local,Loja,Fundacion de Loja,False
342,2017-12-08,Transfer,Local,Quito,Traslado Fundacion de Quito,False
344,2017-12-22,Holiday,Local,Salinas,Cantonizacion de Salinas,False


In [308]:
holidays = holidays.rename(columns={'locale': 'holiday'})

It might be easier to look at creating separate dataframes for National, Local and Regional holidays to better plan the merge. 

In [309]:
holidays_national = holidays[holidays.holiday == 'National']

In [310]:
holidays_national

,date,type,holiday,locale_name,description,transferred
14,2012-08-10,Holiday,National,Ecuador,Primer Grito de Independencia,False
20,2012-10-12,Transfer,National,Ecuador,Traslado Independencia de Guayaquil,False
21,2012-11-02,Holiday,National,Ecuador,Dia de Difuntos,False
22,2012-11-03,Holiday,National,Ecuador,Independencia de Cuenca,False
31,2012-12-21,Additional,National,Ecuador,Navidad-4,False
...,...,...,...,...,...,...
345,2017-12-22,Additional,National,Ecuador,Navidad-3,False
346,2017-12-23,Additional,National,Ecuador,Navidad-2,False
347,2017-12-24,Additional,National,Ecuador,Navidad-1,False
348,2017-12-25,Holiday,National,Ecuador,Navidad,False


In [311]:
holidays_national[holidays_national.duplicated(['date'], keep=False)]

,date,type,holiday,locale_name,description,transferred
35,2012-12-24,Bridge,National,Ecuador,Puente Navidad,False
36,2012-12-24,Additional,National,Ecuador,Navidad-1,False
39,2012-12-31,Bridge,National,Ecuador,Puente Primer dia del ano,False
40,2012-12-31,Additional,National,Ecuador,Primer dia del ano-1,False
156,2014-12-26,Bridge,National,Ecuador,Puente Navidad,False
157,2014-12-26,Additional,National,Ecuador,Navidad+1,False
235,2016-05-01,Holiday,National,Ecuador,Dia del Trabajo,False
236,2016-05-01,Event,National,Ecuador,Terremoto Manabi+15,False
242,2016-05-07,Additional,National,Ecuador,Dia de la Madre-1,False
243,2016-05-07,Event,National,Ecuador,Terremoto Manabi+21,False


Since it would be ideal not to add any additional rows in the train_test dataframe as a result of the merge, duplicate dates were removed. 

In [312]:
holidays_national = holidays_national.drop_duplicates(subset=['date'], keep='first')

In [313]:
holidays_national[holidays_national.duplicated(['date'], keep=False)]

,date,type,holiday,locale_name,description,transferred


In [314]:
holidays_national

,date,type,holiday,locale_name,description,transferred
14,2012-08-10,Holiday,National,Ecuador,Primer Grito de Independencia,False
20,2012-10-12,Transfer,National,Ecuador,Traslado Independencia de Guayaquil,False
21,2012-11-02,Holiday,National,Ecuador,Dia de Difuntos,False
22,2012-11-03,Holiday,National,Ecuador,Independencia de Cuenca,False
31,2012-12-21,Additional,National,Ecuador,Navidad-4,False
...,...,...,...,...,...,...
345,2017-12-22,Additional,National,Ecuador,Navidad-3,False
346,2017-12-23,Additional,National,Ecuador,Navidad-2,False
347,2017-12-24,Additional,National,Ecuador,Navidad-1,False
348,2017-12-25,Holiday,National,Ecuador,Navidad,False


For further preparation for the merge, type was changed to national_day and all the other columns except date were dropped. 

In [315]:
holidays_national = holidays_national.rename(columns={'type': 'national_day'})

In [316]:
holidays_national['national_day'] = np.where(holidays_national['national_day']=='None', 0, 1)

In [317]:
holidays_national.drop(['holiday', 'locale_name', 'description', 'transferred'], axis=1, inplace=True)

In [318]:
holidays_national

,date,national_day
14,2012-08-10,1
20,2012-10-12,1
21,2012-11-02,1
22,2012-11-03,1
31,2012-12-21,1
...,...,...
345,2017-12-22,1
346,2017-12-23,1
347,2017-12-24,1
348,2017-12-25,1


Unexpectantly there are a number of dates that are in the holidays_national dataframe that are not in the train_test dataframe. This was tested with 2 dates in holidays_national and not in train_test and one date in train_test.  

In [319]:
dates_tt = set(train_test.date.unique())
dates_nh = set(holidays_national.date.unique())

In [320]:
list(dates_nh-dates_tt)

[numpy.datetime64('2012-08-10T00:00:00.000000000'),
 numpy.datetime64('2012-10-12T00:00:00.000000000'),
 numpy.datetime64('2012-11-02T00:00:00.000000000'),
 numpy.datetime64('2012-12-21T00:00:00.000000000'),
 numpy.datetime64('2012-11-03T00:00:00.000000000'),
 numpy.datetime64('2012-12-22T00:00:00.000000000'),
 numpy.datetime64('2012-12-23T00:00:00.000000000'),
 numpy.datetime64('2012-12-24T00:00:00.000000000'),
 numpy.datetime64('2012-12-25T00:00:00.000000000'),
 numpy.datetime64('2012-12-26T00:00:00.000000000'),
 numpy.datetime64('2012-12-31T00:00:00.000000000'),
 numpy.datetime64('2013-12-25T00:00:00.000000000'),
 numpy.datetime64('2017-12-24T00:00:00.000000000'),
 numpy.datetime64('2017-10-09T00:00:00.000000000'),
 numpy.datetime64('2016-12-25T00:00:00.000000000'),
 numpy.datetime64('2017-12-26T00:00:00.000000000'),
 numpy.datetime64('2017-11-02T00:00:00.000000000'),
 numpy.datetime64('2015-12-25T00:00:00.000000000'),
 numpy.datetime64('2017-12-25T00:00:00.000000000'),
 numpy.datet

In [321]:
train_test.loc[train_test['date'] == '2012-12-25']

,id,date,store_nbr,family,sales,onpromotion,day_of_week,weekend,month,day_of_year,week_of_year,year,dcoilwtico,city,state,type,cluster,transactions


In [322]:
train_test.loc[train_test['date'] == '2012-10-12']

,id,date,store_nbr,family,sales,onpromotion,day_of_week,weekend,month,day_of_year,week_of_year,year,dcoilwtico,city,state,type,cluster,transactions


In [323]:
train_test.loc[train_test['date'] == '2017-08-31']

,id,date,store_nbr,family,sales,onpromotion,day_of_week,weekend,month,day_of_year,week_of_year,year,dcoilwtico,city,state,type,cluster,transactions
56067,3027618,2017-08-31,1,AUTOMOTIVE,NaN,0,4.0,0.0,8.0,243.0,35,2017.0,47.26,Quito,Pichincha,D,13,0.0
56068,3027619,2017-08-31,1,BABY CARE,NaN,0,4.0,0.0,8.0,243.0,35,2017.0,47.26,Quito,Pichincha,D,13,0.0
56069,3027620,2017-08-31,1,BEAUTY,NaN,1,4.0,0.0,8.0,243.0,35,2017.0,47.26,Quito,Pichincha,D,13,0.0
56070,3027621,2017-08-31,1,BEVERAGES,NaN,33,4.0,0.0,8.0,243.0,35,2017.0,47.26,Quito,Pichincha,D,13,0.0
56071,3027622,2017-08-31,1,BOOKS,NaN,0,4.0,0.0,8.0,243.0,35,2017.0,47.26,Quito,Pichincha,D,13,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3029395,3029395,2017-08-31,9,POULTRY,NaN,1,4.0,0.0,8.0,243.0,35,2017.0,47.26,Quito,Pichincha,B,6,0.0
3029396,3029396,2017-08-31,9,PREPARED FOODS,NaN,0,4.0,0.0,8.0,243.0,35,2017.0,47.26,Quito,Pichincha,B,6,0.0
3029397,3029397,2017-08-31,9,PRODUCE,NaN,1,4.0,0.0,8.0,243.0,35,2017.0,47.26,Quito,Pichincha,B,6,0.0
3029398,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,NaN,9,4.0,0.0,8.0,243.0,35,2017.0,47.26,Quito,Pichincha,B,6,0.0


In [324]:
train_test = pd.merge(train_test, holidays_national, on='date', how='outer')

In [325]:
train_test

,id,date,store_nbr,family,sales,onpromotion,day_of_week,weekend,month,day_of_year,week_of_year,year,dcoilwtico,city,state,type,cluster,transactions,national_day
0,0.0,2013-01-01,1.0,AUTOMOTIVE,0.0,0.0,2.0,0.0,1.0,1.0,1,2013.0,0.0,Quito,Pichincha,D,13.0,0.0,1.0
1,1.0,2013-01-01,1.0,BABY CARE,0.0,0.0,2.0,0.0,1.0,1.0,1,2013.0,0.0,Quito,Pichincha,D,13.0,0.0,1.0
2,2.0,2013-01-01,1.0,BEAUTY,0.0,0.0,2.0,0.0,1.0,1.0,1,2013.0,0.0,Quito,Pichincha,D,13.0,0.0,1.0
3,3.0,2013-01-01,1.0,BEVERAGES,0.0,0.0,2.0,0.0,1.0,1.0,1,2013.0,0.0,Quito,Pichincha,D,13.0,0.0,1.0
4,4.0,2013-01-01,1.0,BOOKS,0.0,0.0,2.0,0.0,1.0,1.0,1,2013.0,0.0,Quito,Pichincha,D,13.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3029419,NaN,2017-12-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3029420,NaN,2017-12-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3029421,NaN,2017-12-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3029422,NaN,2017-12-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [326]:
train_test = train_test[train_test['id'].notna()]

In [327]:
train_test

,id,date,store_nbr,family,sales,onpromotion,day_of_week,weekend,month,day_of_year,week_of_year,year,dcoilwtico,city,state,type,cluster,transactions,national_day
0,0.0,2013-01-01,1.0,AUTOMOTIVE,0.0,0.0,2.0,0.0,1.0,1.0,1,2013.0,0.00,Quito,Pichincha,D,13.0,0.0,1.0
1,1.0,2013-01-01,1.0,BABY CARE,0.0,0.0,2.0,0.0,1.0,1.0,1,2013.0,0.00,Quito,Pichincha,D,13.0,0.0,1.0
2,2.0,2013-01-01,1.0,BEAUTY,0.0,0.0,2.0,0.0,1.0,1.0,1,2013.0,0.00,Quito,Pichincha,D,13.0,0.0,1.0
3,3.0,2013-01-01,1.0,BEVERAGES,0.0,0.0,2.0,0.0,1.0,1.0,1,2013.0,0.00,Quito,Pichincha,D,13.0,0.0,1.0
4,4.0,2013-01-01,1.0,BOOKS,0.0,0.0,2.0,0.0,1.0,1.0,1,2013.0,0.00,Quito,Pichincha,D,13.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3029395,3029395.0,2017-08-31,9.0,POULTRY,NaN,1.0,4.0,0.0,8.0,243.0,35,2017.0,47.26,Quito,Pichincha,B,6.0,0.0,NaN
3029396,3029396.0,2017-08-31,9.0,PREPARED FOODS,NaN,0.0,4.0,0.0,8.0,243.0,35,2017.0,47.26,Quito,Pichincha,B,6.0,0.0,NaN
3029397,3029397.0,2017-08-31,9.0,PRODUCE,NaN,1.0,4.0,0.0,8.0,243.0,35,2017.0,47.26,Quito,Pichincha,B,6.0,0.0,NaN
3029398,3029398.0,2017-08-31,9.0,SCHOOL AND OFFICE SUPPLIES,NaN,9.0,4.0,0.0,8.0,243.0,35,2017.0,47.26,Quito,Pichincha,B,6.0,0.0,NaN


In [328]:
train_test.dtypes

id                     float64
date            datetime64[ns]
store_nbr              float64
family                  object
sales                  float64
onpromotion            float64
day_of_week            float64
weekend                float64
month                  float64
day_of_year            float64
week_of_year            UInt32
year                   float64
dcoilwtico             float64
city                    object
state                   object
type                    object
cluster                float64
transactions           float64
national_day           float64
dtype: object

To clean up the formatting of the float variables. 

In [329]:
pd.options.mode.chained_assignment = None 

In [330]:
train_test['id']=train_test['id'].astype(np.int64)
train_test['store_nbr']=train_test['store_nbr'].astype(np.int64)
train_test['onpromotion']=train_test['onpromotion'].astype(np.int64)
train_test['cluster']=train_test['cluster'].astype(np.int64)
train_test['day_of_week']=train_test['day_of_week'].astype(np.int64)
train_test['weekend']=train_test['weekend'].astype(np.int64)
train_test['month']=train_test['month'].astype(np.int64)
train_test['day_of_year']=train_test['day_of_year'].astype(np.int64)
train_test['week_of_year']=train_test['week_of_year'].astype(np.int64)
train_test['year']=train_test['year'].astype(np.int64)

In [331]:
train_test

,id,date,store_nbr,family,sales,onpromotion,day_of_week,weekend,month,day_of_year,week_of_year,year,dcoilwtico,city,state,type,cluster,transactions,national_day
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,2,0,1,1,1,2013,0.00,Quito,Pichincha,D,13,0.0,1.0
1,1,2013-01-01,1,BABY CARE,0.0,0,2,0,1,1,1,2013,0.00,Quito,Pichincha,D,13,0.0,1.0
2,2,2013-01-01,1,BEAUTY,0.0,0,2,0,1,1,1,2013,0.00,Quito,Pichincha,D,13,0.0,1.0
3,3,2013-01-01,1,BEVERAGES,0.0,0,2,0,1,1,1,2013,0.00,Quito,Pichincha,D,13,0.0,1.0
4,4,2013-01-01,1,BOOKS,0.0,0,2,0,1,1,1,2013,0.00,Quito,Pichincha,D,13,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3029395,3029395,2017-08-31,9,POULTRY,NaN,1,4,0,8,243,35,2017,47.26,Quito,Pichincha,B,6,0.0,NaN
3029396,3029396,2017-08-31,9,PREPARED FOODS,NaN,0,4,0,8,243,35,2017,47.26,Quito,Pichincha,B,6,0.0,NaN
3029397,3029397,2017-08-31,9,PRODUCE,NaN,1,4,0,8,243,35,2017,47.26,Quito,Pichincha,B,6,0.0,NaN
3029398,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,NaN,9,4,0,8,243,35,2017,47.26,Quito,Pichincha,B,6,0.0,NaN


Now a new regional holidays dataframe is created. There are no duplicate dates in this dataframe. 

In [332]:
holidays_regional = holidays[holidays.holiday == 'Regional']

In [333]:
holidays_regional

,date,type,holiday,locale_name,description,transferred
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
7,2012-06-25,Holiday,Regional,Imbabura,Provincializacion de Imbabura,False
23,2012-11-06,Holiday,Regional,Santo Domingo de los Tsachilas,Provincializacion de Santo Domingo,False
24,2012-11-07,Holiday,Regional,Santa Elena,Provincializacion Santa Elena,False
47,2013-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
58,2013-06-25,Holiday,Regional,Imbabura,Provincializacion de Imbabura,False
76,2013-11-06,Holiday,Regional,Santo Domingo de los Tsachilas,Provincializacion de Santo Domingo,False
77,2013-11-07,Holiday,Regional,Santa Elena,Provincializacion Santa Elena,False
96,2014-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
112,2014-06-25,Holiday,Regional,Imbabura,Provincializacion de Imbabura,False


In [334]:
holidays_regional[holidays_regional.duplicated(['date'], keep=False)]

,date,type,holiday,locale_name,description,transferred


In [335]:
region_tt = set(train_test.state.unique())
region_rh = set(holidays_regional.locale_name.unique())

In [336]:
region_tt

{'Azuay',
 'Bolivar',
 'Chimborazo',
 'Cotopaxi',
 'El Oro',
 'Esmeraldas',
 'Guayas',
 'Imbabura',
 'Loja',
 'Los Rios',
 'Manabi',
 'Pastaza',
 'Pichincha',
 'Santa Elena',
 'Santo Domingo de los Tsachilas',
 'Tungurahua'}

In [337]:
region_rh

{'Cotopaxi', 'Imbabura', 'Santa Elena', 'Santo Domingo de los Tsachilas'}

In [338]:
holidays_regional = holidays_regional.rename(columns={'type': 'regional_day', 'locale_name': 'state'})

In [339]:
holidays_regional['regional_day'] = np.where(holidays_regional['regional_day']=='None', 0, 1)

In [340]:
holidays_regional

,date,regional_day,holiday,state,description,transferred
1,2012-04-01,1,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
7,2012-06-25,1,Regional,Imbabura,Provincializacion de Imbabura,False
23,2012-11-06,1,Regional,Santo Domingo de los Tsachilas,Provincializacion de Santo Domingo,False
24,2012-11-07,1,Regional,Santa Elena,Provincializacion Santa Elena,False
47,2013-04-01,1,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
58,2013-06-25,1,Regional,Imbabura,Provincializacion de Imbabura,False
76,2013-11-06,1,Regional,Santo Domingo de los Tsachilas,Provincializacion de Santo Domingo,False
77,2013-11-07,1,Regional,Santa Elena,Provincializacion Santa Elena,False
96,2014-04-01,1,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
112,2014-06-25,1,Regional,Imbabura,Provincializacion de Imbabura,False


In [341]:
holidays_regional.drop(['holiday', 'description', 'transferred'], axis=1, inplace=True)

In [342]:
holidays_regional

,date,regional_day,state
1,2012-04-01,1,Cotopaxi
7,2012-06-25,1,Imbabura
23,2012-11-06,1,Santo Domingo de los Tsachilas
24,2012-11-07,1,Santa Elena
47,2013-04-01,1,Cotopaxi
58,2013-06-25,1,Imbabura
76,2013-11-06,1,Santo Domingo de los Tsachilas
77,2013-11-07,1,Santa Elena
96,2014-04-01,1,Cotopaxi
112,2014-06-25,1,Imbabura


There appear to be 6 dates that are in the holidays_regional dataframe but not in the train_test dataframe. These rows were then eliminated. 

In [343]:
dates_tt = set(train_test.date.unique())
dates_rh = set(holidays_regional.date.unique())

In [344]:
list(dates_rh-dates_tt)

[numpy.datetime64('2012-04-01T00:00:00.000000000'),
 numpy.datetime64('2012-06-25T00:00:00.000000000'),
 numpy.datetime64('2012-11-06T00:00:00.000000000'),
 numpy.datetime64('2012-11-07T00:00:00.000000000'),
 numpy.datetime64('2017-11-06T00:00:00.000000000'),
 numpy.datetime64('2017-11-07T00:00:00.000000000')]

In [345]:
regional_dates = list(dates_rh-dates_tt)

In [346]:
holidays_regional = holidays_regional[~holidays_regional.date.isin(regional_dates)]

In [347]:
holidays_regional

,date,regional_day,state
47,2013-04-01,1,Cotopaxi
58,2013-06-25,1,Imbabura
76,2013-11-06,1,Santo Domingo de los Tsachilas
77,2013-11-07,1,Santa Elena
96,2014-04-01,1,Cotopaxi
112,2014-06-25,1,Imbabura
139,2014-11-06,1,Santo Domingo de los Tsachilas
140,2014-11-07,1,Santa Elena
165,2015-04-01,1,Cotopaxi
177,2015-06-25,1,Imbabura


To merge based on date and then state. 

In [348]:
train_test_regional = pd.merge(train_test, holidays_regional, on=['date', 'state'], how='outer')

In [349]:
train_test_regional

,id,date,store_nbr,family,sales,onpromotion,day_of_week,weekend,month,day_of_year,week_of_year,year,dcoilwtico,city,state,type,cluster,transactions,national_day,regional_day
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,2,0,1,1,1,2013,0.00,Quito,Pichincha,D,13,0.0,1.0,NaN
1,1,2013-01-01,1,BABY CARE,0.0,0,2,0,1,1,1,2013,0.00,Quito,Pichincha,D,13,0.0,1.0,NaN
2,2,2013-01-01,1,BEAUTY,0.0,0,2,0,1,1,1,2013,0.00,Quito,Pichincha,D,13,0.0,1.0,NaN
3,3,2013-01-01,1,BEVERAGES,0.0,0,2,0,1,1,1,2013,0.00,Quito,Pichincha,D,13,0.0,1.0,NaN
4,4,2013-01-01,1,BOOKS,0.0,0,2,0,1,1,1,2013,0.00,Quito,Pichincha,D,13,0.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3029395,3029263,2017-08-31,54,POULTRY,NaN,0,4,0,8,243,35,2017,47.26,El Carmen,Manabi,C,3,0.0,NaN,NaN
3029396,3029264,2017-08-31,54,PREPARED FOODS,NaN,0,4,0,8,243,35,2017,47.26,El Carmen,Manabi,C,3,0.0,NaN,NaN
3029397,3029265,2017-08-31,54,PRODUCE,NaN,1,4,0,8,243,35,2017,47.26,El Carmen,Manabi,C,3,0.0,NaN,NaN
3029398,3029266,2017-08-31,54,SCHOOL AND OFFICE SUPPLIES,NaN,0,4,0,8,243,35,2017,47.26,El Carmen,Manabi,C,3,0.0,NaN,NaN


The last id number above doesn't match the test dataframe, so the train_test_regional was resorted to ensure the last values matched the test dataframe. 

In [350]:
test

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0
...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1
28508,3029396,2017-08-31,9,PREPARED FOODS,0
28509,3029397,2017-08-31,9,PRODUCE,1
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9


In [351]:
train_test_regional.sort_values('id')

,id,date,store_nbr,family,sales,onpromotion,day_of_week,weekend,month,day_of_year,week_of_year,year,dcoilwtico,city,state,type,cluster,transactions,national_day,regional_day
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,2,0,1,1,1,2013,0.00,Quito,Pichincha,D,13,0.0,1.0,NaN
1,1,2013-01-01,1,BABY CARE,0.0,0,2,0,1,1,1,2013,0.00,Quito,Pichincha,D,13,0.0,1.0,NaN
2,2,2013-01-01,1,BEAUTY,0.0,0,2,0,1,1,1,2013,0.00,Quito,Pichincha,D,13,0.0,1.0,NaN
3,3,2013-01-01,1,BEVERAGES,0.0,0,2,0,1,1,1,2013,0.00,Quito,Pichincha,D,13,0.0,1.0,NaN
4,4,2013-01-01,1,BOOKS,0.0,0,2,0,1,1,1,2013,0.00,Quito,Pichincha,D,13,0.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3028240,3029395,2017-08-31,9,POULTRY,NaN,1,4,0,8,243,35,2017,47.26,Quito,Pichincha,B,6,0.0,NaN,NaN
3028241,3029396,2017-08-31,9,PREPARED FOODS,NaN,0,4,0,8,243,35,2017,47.26,Quito,Pichincha,B,6,0.0,NaN,NaN
3028242,3029397,2017-08-31,9,PRODUCE,NaN,1,4,0,8,243,35,2017,47.26,Quito,Pichincha,B,6,0.0,NaN,NaN
3028243,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,NaN,9,4,0,8,243,35,2017,47.26,Quito,Pichincha,B,6,0.0,NaN,NaN


To test to make sure the regional holidays are showing up by date and state. 

In [352]:
train_test_regional.loc[(train_test_regional['date'] == '2013-04-01') & (train_test_regional['state'] == 'Cotopaxi')]

,id,date,store_nbr,family,sales,onpromotion,day_of_week,weekend,month,day_of_year,week_of_year,year,dcoilwtico,city,state,type,cluster,transactions,national_day,regional_day
161007,160479,2013-04-01,12,AUTOMOTIVE,3.000,0,1,0,4,91,14,2013,97.1,Latacunga,Cotopaxi,C,15,1313.0,NaN,1.0
161008,160480,2013-04-01,12,BABY CARE,0.000,0,1,0,4,91,14,2013,97.1,Latacunga,Cotopaxi,C,15,1313.0,NaN,1.0
161009,160481,2013-04-01,12,BEAUTY,4.000,0,1,0,4,91,14,2013,97.1,Latacunga,Cotopaxi,C,15,1313.0,NaN,1.0
161010,160482,2013-04-01,12,BEVERAGES,762.000,0,1,0,4,91,14,2013,97.1,Latacunga,Cotopaxi,C,15,1313.0,NaN,1.0
161011,160483,2013-04-01,12,BOOKS,0.000,0,1,0,4,91,14,2013,97.1,Latacunga,Cotopaxi,C,15,1313.0,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161068,160540,2013-04-01,13,POULTRY,85.639,0,1,0,4,91,14,2013,97.1,Latacunga,Cotopaxi,C,15,1125.0,NaN,1.0
161069,160541,2013-04-01,13,PREPARED FOODS,20.000,0,1,0,4,91,14,2013,97.1,Latacunga,Cotopaxi,C,15,1125.0,NaN,1.0
161070,160542,2013-04-01,13,PRODUCE,0.000,0,1,0,4,91,14,2013,97.1,Latacunga,Cotopaxi,C,15,1125.0,NaN,1.0
161071,160543,2013-04-01,13,SCHOOL AND OFFICE SUPPLIES,0.000,0,1,0,4,91,14,2013,97.1,Latacunga,Cotopaxi,C,15,1125.0,NaN,1.0


In [353]:
train_test = train_test_regional

In [354]:
train_test

,id,date,store_nbr,family,sales,onpromotion,day_of_week,weekend,month,day_of_year,week_of_year,year,dcoilwtico,city,state,type,cluster,transactions,national_day,regional_day
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,2,0,1,1,1,2013,0.00,Quito,Pichincha,D,13,0.0,1.0,NaN
1,1,2013-01-01,1,BABY CARE,0.0,0,2,0,1,1,1,2013,0.00,Quito,Pichincha,D,13,0.0,1.0,NaN
2,2,2013-01-01,1,BEAUTY,0.0,0,2,0,1,1,1,2013,0.00,Quito,Pichincha,D,13,0.0,1.0,NaN
3,3,2013-01-01,1,BEVERAGES,0.0,0,2,0,1,1,1,2013,0.00,Quito,Pichincha,D,13,0.0,1.0,NaN
4,4,2013-01-01,1,BOOKS,0.0,0,2,0,1,1,1,2013,0.00,Quito,Pichincha,D,13,0.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3029395,3029263,2017-08-31,54,POULTRY,NaN,0,4,0,8,243,35,2017,47.26,El Carmen,Manabi,C,3,0.0,NaN,NaN
3029396,3029264,2017-08-31,54,PREPARED FOODS,NaN,0,4,0,8,243,35,2017,47.26,El Carmen,Manabi,C,3,0.0,NaN,NaN
3029397,3029265,2017-08-31,54,PRODUCE,NaN,1,4,0,8,243,35,2017,47.26,El Carmen,Manabi,C,3,0.0,NaN,NaN
3029398,3029266,2017-08-31,54,SCHOOL AND OFFICE SUPPLIES,NaN,0,4,0,8,243,35,2017,47.26,El Carmen,Manabi,C,3,0.0,NaN,NaN


A new local holidays dataframe is created. 

In [355]:
holidays_local = holidays[holidays.holiday == 'Local']

In [356]:
holidays_local

,date,type,holiday,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
5,2012-05-12,Holiday,Local,Puyo,Cantonizacion del Puyo,False
...,...,...,...,...,...,...
338,2017-11-12,Holiday,Local,Ambato,Independencia de Ambato,False
339,2017-12-05,Additional,Local,Quito,Fundacion de Quito-1,False
341,2017-12-08,Holiday,Local,Loja,Fundacion de Loja,False
342,2017-12-08,Transfer,Local,Quito,Traslado Fundacion de Quito,False


To explore local holidays with similar dates. All of these have separate cities except for 2016-07-24 for Guayaquil. That could cause a problem in merging into the train_test dataframe since their won't be a unique value each for date and city. So the 'Transfer' locale_day for that date will be deleted. 

In [357]:
holidays_local[holidays_local.duplicated(['date'], keep=False)]

,date,type,holiday,locale_name,description,transferred
8,2012-06-25,Holiday,Local,Latacunga,Cantonizacion de Latacunga,False
9,2012-06-25,Holiday,Local,Machala,Fundacion de Machala,False
10,2012-07-03,Holiday,Local,Santo Domingo,Fundacion de Santo Domingo,False
11,2012-07-03,Holiday,Local,El Carmen,Cantonizacion de El Carmen,False
59,2013-06-25,Holiday,Local,Machala,Fundacion de Machala,False
60,2013-06-25,Holiday,Local,Latacunga,Cantonizacion de Latacunga,False
61,2013-07-03,Holiday,Local,El Carmen,Cantonizacion de El Carmen,False
62,2013-07-03,Holiday,Local,Santo Domingo,Fundacion de Santo Domingo,False
110,2014-06-25,Holiday,Local,Latacunga,Cantonizacion de Latacunga,False
111,2014-06-25,Holiday,Local,Machala,Fundacion de Machala,False


In [358]:
holidays_local = holidays_local.drop(holidays_local[(holidays_local['locale_name'] == 'Guayaquil') & 
                                                    (holidays_local['type'] == 'Transfer')].index)

In [359]:
holidays_local

,date,type,holiday,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
5,2012-05-12,Holiday,Local,Puyo,Cantonizacion del Puyo,False
...,...,...,...,...,...,...
338,2017-11-12,Holiday,Local,Ambato,Independencia de Ambato,False
339,2017-12-05,Additional,Local,Quito,Fundacion de Quito-1,False
341,2017-12-08,Holiday,Local,Loja,Fundacion de Loja,False
342,2017-12-08,Transfer,Local,Quito,Traslado Fundacion de Quito,False


For local holidays, all the values in the holidays_local dataframe are also in the train_test dataframe. There are 3 cities in the train_test dataframe that are not in the holidays_local dataframe. 

In [360]:
local_tt = set(train_test.city.unique())
local_lh = set(holidays_local.locale_name.unique())

In [361]:
local_tt

{'Ambato',
 'Babahoyo',
 'Cayambe',
 'Cuenca',
 'Daule',
 'El Carmen',
 'Esmeraldas',
 'Guaranda',
 'Guayaquil',
 'Ibarra',
 'Latacunga',
 'Libertad',
 'Loja',
 'Machala',
 'Manta',
 'Playas',
 'Puyo',
 'Quevedo',
 'Quito',
 'Riobamba',
 'Salinas',
 'Santo Domingo'}

In [362]:
local_lh

{'Ambato',
 'Cayambe',
 'Cuenca',
 'El Carmen',
 'Esmeraldas',
 'Guaranda',
 'Guayaquil',
 'Ibarra',
 'Latacunga',
 'Libertad',
 'Loja',
 'Machala',
 'Manta',
 'Puyo',
 'Quevedo',
 'Quito',
 'Riobamba',
 'Salinas',
 'Santo Domingo'}

In [363]:
list(local_tt-local_lh)

['Daule', 'Playas', 'Babahoyo']

In [364]:
list(local_lh-local_tt)

[]

To set up the holidays_local dataframe for merging into train_test. 

In [365]:
holidays_local = holidays_local.rename(columns={'type': 'local_day', 'locale_name': 'city'})

In [366]:
holidays_local['local_day'] = np.where(holidays_local['local_day']=='None', 0, 1)

In [367]:
holidays_local.drop(['holiday', 'description', 'transferred'], axis=1, inplace=True)

In [368]:
holidays_local

,date,local_day,city
0,2012-03-02,1,Manta
2,2012-04-12,1,Cuenca
3,2012-04-14,1,Libertad
4,2012-04-21,1,Riobamba
5,2012-05-12,1,Puyo
...,...,...,...
338,2017-11-12,1,Ambato
339,2017-12-05,1,Quito
341,2017-12-08,1,Loja
342,2017-12-08,1,Quito


There are a number of dates in holidays_local that are not in train_test. These rows are eliminated. 

In [369]:
dates_tt = set(train_test.date.unique())
dates_lh = set(holidays_local.date.unique())

In [370]:
list(dates_lh-dates_tt)

[numpy.datetime64('2012-03-02T00:00:00.000000000'),
 numpy.datetime64('2012-04-12T00:00:00.000000000'),
 numpy.datetime64('2012-04-21T00:00:00.000000000'),
 numpy.datetime64('2012-05-12T00:00:00.000000000'),
 numpy.datetime64('2012-04-14T00:00:00.000000000'),
 numpy.datetime64('2012-06-23T00:00:00.000000000'),
 numpy.datetime64('2012-06-25T00:00:00.000000000'),
 numpy.datetime64('2012-07-03T00:00:00.000000000'),
 numpy.datetime64('2012-07-23T00:00:00.000000000'),
 numpy.datetime64('2012-08-05T00:00:00.000000000'),
 numpy.datetime64('2012-08-15T00:00:00.000000000'),
 numpy.datetime64('2012-08-24T00:00:00.000000000'),
 numpy.datetime64('2012-11-12T00:00:00.000000000'),
 numpy.datetime64('2012-12-06T00:00:00.000000000'),
 numpy.datetime64('2017-10-07T00:00:00.000000000'),
 numpy.datetime64('2017-11-10T00:00:00.000000000'),
 numpy.datetime64('2012-11-10T00:00:00.000000000'),
 numpy.datetime64('2012-10-07T00:00:00.000000000'),
 numpy.datetime64('2017-09-29T00:00:00.000000000'),
 numpy.datet

In [371]:
local_dates = list(dates_lh-dates_tt)

In [372]:
len(local_dates)

29

In [373]:
holidays_local = holidays_local[~holidays_local.date.isin(local_dates)]

In [374]:
holidays_local

,date,local_day,city
46,2013-03-02,1,Manta
48,2013-04-12,1,Cuenca
49,2013-04-14,1,Libertad
50,2013-04-21,1,Riobamba
54,2013-05-12,1,Puyo
...,...,...,...
321,2017-07-24,1,Guayaquil
322,2017-07-25,1,Guayaquil
323,2017-08-05,1,Esmeraldas
326,2017-08-15,1,Riobamba


To create a new train_test_local dataframe with the merger of the train_test and holidays_local dataframes on city and date. 

In [375]:
train_test_local = pd.merge(train_test, holidays_local, on=['city', 'date'], how='outer')

In [376]:
train_test_local

,id,date,store_nbr,family,sales,onpromotion,day_of_week,weekend,month,day_of_year,...,year,dcoilwtico,city,state,type,cluster,transactions,national_day,regional_day,local_day
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,2,0,1,1,...,2013,0.00,Quito,Pichincha,D,13,0.0,1.0,NaN,NaN
1,1,2013-01-01,1,BABY CARE,0.0,0,2,0,1,1,...,2013,0.00,Quito,Pichincha,D,13,0.0,1.0,NaN,NaN
2,2,2013-01-01,1,BEAUTY,0.0,0,2,0,1,1,...,2013,0.00,Quito,Pichincha,D,13,0.0,1.0,NaN,NaN
3,3,2013-01-01,1,BEVERAGES,0.0,0,2,0,1,1,...,2013,0.00,Quito,Pichincha,D,13,0.0,1.0,NaN,NaN
4,4,2013-01-01,1,BOOKS,0.0,0,2,0,1,1,...,2013,0.00,Quito,Pichincha,D,13,0.0,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3029395,3029263,2017-08-31,54,POULTRY,NaN,0,4,0,8,243,...,2017,47.26,El Carmen,Manabi,C,3,0.0,NaN,NaN,NaN
3029396,3029264,2017-08-31,54,PREPARED FOODS,NaN,0,4,0,8,243,...,2017,47.26,El Carmen,Manabi,C,3,0.0,NaN,NaN,NaN
3029397,3029265,2017-08-31,54,PRODUCE,NaN,1,4,0,8,243,...,2017,47.26,El Carmen,Manabi,C,3,0.0,NaN,NaN,NaN
3029398,3029266,2017-08-31,54,SCHOOL AND OFFICE SUPPLIES,NaN,0,4,0,8,243,...,2017,47.26,El Carmen,Manabi,C,3,0.0,NaN,NaN,NaN


To sort the new dataframe to ensure the last id's match the original dataframe id's. 

In [377]:
train_test_local.sort_values('id')

,id,date,store_nbr,family,sales,onpromotion,day_of_week,weekend,month,day_of_year,...,year,dcoilwtico,city,state,type,cluster,transactions,national_day,regional_day,local_day
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,2,0,1,1,...,2013,0.00,Quito,Pichincha,D,13,0.0,1.0,NaN,NaN
1,1,2013-01-01,1,BABY CARE,0.0,0,2,0,1,1,...,2013,0.00,Quito,Pichincha,D,13,0.0,1.0,NaN,NaN
2,2,2013-01-01,1,BEAUTY,0.0,0,2,0,1,1,...,2013,0.00,Quito,Pichincha,D,13,0.0,1.0,NaN,NaN
3,3,2013-01-01,1,BEVERAGES,0.0,0,2,0,1,1,...,2013,0.00,Quito,Pichincha,D,13,0.0,1.0,NaN,NaN
4,4,2013-01-01,1,BOOKS,0.0,0,2,0,1,1,...,2013,0.00,Quito,Pichincha,D,13,0.0,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3028207,3029395,2017-08-31,9,POULTRY,NaN,1,4,0,8,243,...,2017,47.26,Quito,Pichincha,B,6,0.0,NaN,NaN,NaN
3028208,3029396,2017-08-31,9,PREPARED FOODS,NaN,0,4,0,8,243,...,2017,47.26,Quito,Pichincha,B,6,0.0,NaN,NaN,NaN
3028209,3029397,2017-08-31,9,PRODUCE,NaN,1,4,0,8,243,...,2017,47.26,Quito,Pichincha,B,6,0.0,NaN,NaN,NaN
3028210,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,NaN,9,4,0,8,243,...,2017,47.26,Quito,Pichincha,B,6,0.0,NaN,NaN,NaN


The last thing to do is replace the NaNs in the 3 new holidays columns with 'None'. 

In [383]:
train_test_local['national_day'].fillna(0, inplace=True)
train_test_local['regional_day'].fillna(0, inplace=True)
train_test_local['local_day'].fillna(0, inplace=True)

In [384]:
train_test_local

,id,date,store_nbr,family,sales,onpromotion,day_of_week,weekend,month,day_of_year,...,year,dcoilwtico,city,state,type,cluster,transactions,national_day,regional_day,local_day
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,2,0,1,1,...,2013,0.00,Quito,Pichincha,D,13,0.0,1.0,0.0,0.0
1,1,2013-01-01,1,BABY CARE,0.0,0,2,0,1,1,...,2013,0.00,Quito,Pichincha,D,13,0.0,1.0,0.0,0.0
2,2,2013-01-01,1,BEAUTY,0.0,0,2,0,1,1,...,2013,0.00,Quito,Pichincha,D,13,0.0,1.0,0.0,0.0
3,3,2013-01-01,1,BEVERAGES,0.0,0,2,0,1,1,...,2013,0.00,Quito,Pichincha,D,13,0.0,1.0,0.0,0.0
4,4,2013-01-01,1,BOOKS,0.0,0,2,0,1,1,...,2013,0.00,Quito,Pichincha,D,13,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3029395,3029263,2017-08-31,54,POULTRY,NaN,0,4,0,8,243,...,2017,47.26,El Carmen,Manabi,C,3,0.0,0.0,0.0,0.0
3029396,3029264,2017-08-31,54,PREPARED FOODS,NaN,0,4,0,8,243,...,2017,47.26,El Carmen,Manabi,C,3,0.0,0.0,0.0,0.0
3029397,3029265,2017-08-31,54,PRODUCE,NaN,1,4,0,8,243,...,2017,47.26,El Carmen,Manabi,C,3,0.0,0.0,0.0,0.0
3029398,3029266,2017-08-31,54,SCHOOL AND OFFICE SUPPLIES,NaN,0,4,0,8,243,...,2017,47.26,El Carmen,Manabi,C,3,0.0,0.0,0.0,0.0


In [385]:
train_test = train_test_local

In [386]:
train_test

,id,date,store_nbr,family,sales,onpromotion,day_of_week,weekend,month,day_of_year,...,year,dcoilwtico,city,state,type,cluster,transactions,national_day,regional_day,local_day
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,2,0,1,1,...,2013,0.00,Quito,Pichincha,D,13,0.0,1.0,0.0,0.0
1,1,2013-01-01,1,BABY CARE,0.0,0,2,0,1,1,...,2013,0.00,Quito,Pichincha,D,13,0.0,1.0,0.0,0.0
2,2,2013-01-01,1,BEAUTY,0.0,0,2,0,1,1,...,2013,0.00,Quito,Pichincha,D,13,0.0,1.0,0.0,0.0
3,3,2013-01-01,1,BEVERAGES,0.0,0,2,0,1,1,...,2013,0.00,Quito,Pichincha,D,13,0.0,1.0,0.0,0.0
4,4,2013-01-01,1,BOOKS,0.0,0,2,0,1,1,...,2013,0.00,Quito,Pichincha,D,13,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3029395,3029263,2017-08-31,54,POULTRY,NaN,0,4,0,8,243,...,2017,47.26,El Carmen,Manabi,C,3,0.0,0.0,0.0,0.0
3029396,3029264,2017-08-31,54,PREPARED FOODS,NaN,0,4,0,8,243,...,2017,47.26,El Carmen,Manabi,C,3,0.0,0.0,0.0,0.0
3029397,3029265,2017-08-31,54,PRODUCE,NaN,1,4,0,8,243,...,2017,47.26,El Carmen,Manabi,C,3,0.0,0.0,0.0,0.0
3029398,3029266,2017-08-31,54,SCHOOL AND OFFICE SUPPLIES,NaN,0,4,0,8,243,...,2017,47.26,El Carmen,Manabi,C,3,0.0,0.0,0.0,0.0


To confirm one last time that the only missing values are the expected sales values from the original test dataframe. 

In [387]:
pd.isnull(train_test).sum()[pd.isnull(train_test).sum() > 0]

sales    28512
dtype: int64

To double check there are still a valid number of holidays listed. 

In [393]:
train_test.national_day.value_counts() 

0.0    2787048
1.0     242352
Name: national_day, dtype: int64

In [394]:
train_test.regional_day.value_counts() 

0.0    3028377
1.0       1023
Name: regional_day, dtype: int64

In [395]:
train_test.local_day.value_counts() 

0.0    3017454
1.0      11946
Name: local_day, dtype: int64

To split the train and test dataframes back out. 

In [388]:
train_features = train_test[train_test['sales'].notnull()].copy()
test_features = train_test[train_test['sales'].isnull()].drop(['sales'],axis=1)

In [389]:
train_features

,id,date,store_nbr,family,sales,onpromotion,day_of_week,weekend,month,day_of_year,...,year,dcoilwtico,city,state,type,cluster,transactions,national_day,regional_day,local_day
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,2,0,1,1,...,2013,0.00,Quito,Pichincha,D,13,0.0,1.0,0.0,0.0
1,1,2013-01-01,1,BABY CARE,0.000,0,2,0,1,1,...,2013,0.00,Quito,Pichincha,D,13,0.0,1.0,0.0,0.0
2,2,2013-01-01,1,BEAUTY,0.000,0,2,0,1,1,...,2013,0.00,Quito,Pichincha,D,13,0.0,1.0,0.0,0.0
3,3,2013-01-01,1,BEVERAGES,0.000,0,2,0,1,1,...,2013,0.00,Quito,Pichincha,D,13,0.0,1.0,0.0,0.0
4,4,2013-01-01,1,BOOKS,0.000,0,2,0,1,1,...,2013,0.00,Quito,Pichincha,D,13,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3000883,3000751,2017-08-15,54,POULTRY,59.619,0,2,0,8,227,...,2017,47.57,El Carmen,Manabi,C,3,802.0,0.0,0.0,0.0
3000884,3000752,2017-08-15,54,PREPARED FOODS,94.000,0,2,0,8,227,...,2017,47.57,El Carmen,Manabi,C,3,802.0,0.0,0.0,0.0
3000885,3000753,2017-08-15,54,PRODUCE,915.371,76,2,0,8,227,...,2017,47.57,El Carmen,Manabi,C,3,802.0,0.0,0.0,0.0
3000886,3000754,2017-08-15,54,SCHOOL AND OFFICE SUPPLIES,0.000,0,2,0,8,227,...,2017,47.57,El Carmen,Manabi,C,3,802.0,0.0,0.0,0.0


In [390]:
test_features

,id,date,store_nbr,family,onpromotion,day_of_week,weekend,month,day_of_year,week_of_year,year,dcoilwtico,city,state,type,cluster,transactions,national_day,regional_day,local_day
3000888,3000888,2017-08-16,1,AUTOMOTIVE,0,3,0,8,228,33,2017,46.80,Quito,Pichincha,D,13,0.0,0.0,0.0,0.0
3000889,3000889,2017-08-16,1,BABY CARE,0,3,0,8,228,33,2017,46.80,Quito,Pichincha,D,13,0.0,0.0,0.0,0.0
3000890,3000890,2017-08-16,1,BEAUTY,2,3,0,8,228,33,2017,46.80,Quito,Pichincha,D,13,0.0,0.0,0.0,0.0
3000891,3000891,2017-08-16,1,BEVERAGES,20,3,0,8,228,33,2017,46.80,Quito,Pichincha,D,13,0.0,0.0,0.0,0.0
3000892,3000892,2017-08-16,1,BOOKS,0,3,0,8,228,33,2017,46.80,Quito,Pichincha,D,13,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3029395,3029263,2017-08-31,54,POULTRY,0,4,0,8,243,35,2017,47.26,El Carmen,Manabi,C,3,0.0,0.0,0.0,0.0
3029396,3029264,2017-08-31,54,PREPARED FOODS,0,4,0,8,243,35,2017,47.26,El Carmen,Manabi,C,3,0.0,0.0,0.0,0.0
3029397,3029265,2017-08-31,54,PRODUCE,1,4,0,8,243,35,2017,47.26,El Carmen,Manabi,C,3,0.0,0.0,0.0,0.0
3029398,3029266,2017-08-31,54,SCHOOL AND OFFICE SUPPLIES,0,4,0,8,243,35,2017,47.26,El Carmen,Manabi,C,3,0.0,0.0,0.0,0.0


To save the train_features and test_features as new csv files that can be used in the next stage of the project. 

In [391]:
train_features.to_csv('train_features.csv', index=False)
test_features.to_csv('test_features.csv', index=False)